compound family generator

In [1]:
#!/usr/bin/env python3.10
# coding: utf-8

import re
import pandas as pd

import dpd csv as dataframe

In [2]:
df = pd.read_csv ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep="\t", dtype=str)
df = df.fillna("") 
df_length = df.shape[0]

extract compound family list

In [7]:
test1 = df["Family2"] != ""
test2 = df["Meaning IN CONTEXT"] != ""
test3 = df["Pāli Root"] == ""
test4 = df["Metadata"] != "yes"
filter = test1 & test2 & test3 & test4
compound_family_df = df.loc[filter, ["Family2"]]

#make compound family list
compound_family_list = []
compound_family_df["Family2"].str.split()
compound_family_list = compound_family_df["Family2"].to_list()

#flatten list
compound_family_list = [item for sublist in compound_family_list for item in sublist.split(" ")]
with open ("compound_family_list.csv", "w") as write_list:
	for item in compound_family_list:
		write_list.write(item + "\n")

# import compound_family_list as dataframe
compound_family_list = pd.DataFrame(compound_family_list)

#drop dupes and sort
compound_family_list.drop_duplicates(keep="first", inplace=True)
compound_family_list.sort_values([0], ascending=True, inplace=True)
compound_family_list.reset_index(drop=True, inplace=True)




In [4]:
# row count
compound_family_count = compound_family_list.shape[0]

for row in range (compound_family_count): #compound_family_count
    compound_family =  compound_family_list.iloc[row,0]

    if row % 250 == 0:
        print(f"{row} {compound_family}")
    
    with open (f"output/{compound_family}.html", 'w', encoding= "'utf-8") as html:

        test1 = df["Family2"] != ""
        test2 = df["Family2"].str.contains(rf"\b{compound_family}\b")
        test3 = df["Meaning IN CONTEXT"] != ""
        test4 = df["Pāli Root"] == ""
        
        filter = test1 & test2 & test3 & test4
        compound_family_df = df.loc[filter, ["Pāli1", "POS", "Meaning IN CONTEXT"]]

        # html conversion
        length = compound_family_df.shape[0]
      
        html.write(f"""<table class= "table3"><tbody>""")

        for row in range(length):
            cf_pali = compound_family_df.iloc[row, 0]
            cf_pos = compound_family_df.iloc[row, 1]
            cf_english = compound_family_df.iloc[row, 2]
				# print(f"{row_sf}\t{sf_pali}\t({sf_pos})  \t{sf_english}")

            html.write(f"<tr><th>{cf_pali}</th>")
            html.write(f"<td>{cf_pos}</td>")
            html.write(f"<td>{cf_english}</td></tr>")
        html.write(f"</tbody></table>")

0 a
250 anuvattī
500 bhogī
750 dhi
1000 heṭṭhā
1250 kaṇha
1500 lābha
1750 nigamana
2000 pamodita
2250 paṭisaṃvedī
2500 sadisa
2750 saṅgāma
3000 taṃ
3250 ussukka
3500 viruhana
3750 ājīva
